In [1]:
from utils.data import get_data

class toy_args:

    def __init__(self, IC,tmin,tmax,n_trunk,n_branch):
        self.IC = IC
        self.tmin = tmin
        self.tmax = tmax
        self.xmin = -25
        self.xmax = 25
        self.n_trunk = n_trunk
        self.n_branch = n_branch
        self.load_data = False
        self.method = 'full_fourier'
        self.problem = '1d_wave'
        self.save_data = False
        self.col_N = 100
        self.t_res = 1
        self.x_res = 0.1
        self.data_dt = 0.001
        self.data_modes = 100
        self.zero_zero_mode = True

    
IC = {'c': 5, 'type': 'periodic_gp', 'params': {'lengthscale':0.5, 'variance':1.0}}

my_args = toy_args(IC,0,10,10,10)

data = get_data(my_args)



getting data
False
5
(1, 2, 10, 100)
(1, 2, 10, 100)
(1, 2, 10, 100)
(1, 2, 10, 100)
(1, 2, 10, 100)
(1, 2, 10, 100)
(1, 2, 10, 100)
(1, 2, 10, 100)
(1, 2, 10, 100)
(1, 2, 10, 100)
(10, 2, 10, 100)
5
(1, 2, 10, 100)
(1, 2, 10, 100)
(1, 2, 10, 100)
(1, 2, 10, 100)
(1, 2, 10, 100)
(1, 2, 10, 100)
(1, 2, 10, 100)
(1, 2, 10, 100)
(1, 2, 10, 100)
(1, 2, 10, 100)
(10, 2, 10, 100)


In [2]:
print(data[0].x.shape)

torch.Size([10, 2, 100])


In [3]:
print(data[0].y.shape)

torch.Size([10, 2, 10, 100])
